<a href="https://colab.research.google.com/github/20di/TTGD/blob/main/TTGD_New1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **<- Установка и монтирование Google Drive ** { vertical-output: true, display-mode: "form" }
!apt remove --purge python3-libtorrent -y 2>/dev/null || true
!pip uninstall -y libtorrent lbry-libtorrent 2>/dev/null || true
!rm -rf /usr/local/lib/python3.*/dist-packages/*libtorrent*
!pip install --quiet libtorrent

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title **<- Нажми сюда для начала загрузки** { vertical-output: true, display-mode: "form" }
#@markdown <br><center><h1><b>TTGD</b></h1></center>
#@markdown <center><img src='https://www.libtorrent.org/img/logo-color-text.png' height="50" alt="libtorrent-logo"/><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/121px-Python-logo-notext.svg.png' height="50" alt="python-logo"/><img src='https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_48dp.png' height="50" alt="Drive-logo"/></center><br>

Метод_загрузки = "Torrent File" #@param ["Torrent File", "Magnet"]
Magnet_ссылка = "" #@param {type:"string"}
Папка_в_GDrive = "TTGD" #@param {type:"string"}
Метод_сжатия = "NONE" #@param ["NONE", "ZIP", "TAR"]

# ────────────────────────────────────────────────
# Блок загрузки .torrent файла
# ────────────────────────────────────────────────
import libtorrent as lt
import shutil, string, time, re, os
from google.colab import files
from IPython.display import clear_output

if Метод_загрузки == "Torrent File":
    print("Загрузите ваш .torrent файл:")
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("Файл не выбран!")
    torrent_file = list(uploaded.keys())[0]
    with open(torrent_file, 'rb') as f:
        e = lt.bdecode(f.read())
    info = lt.torrent_info(e)
    torrent_params = {'ti': info}
    os.remove(torrent_file)
else:
    torrent_params = None

#@markdown <br><h1><b>Инструкция</b></h1>
#@markdown - **NONE** — файлы переносятся без сжатия
#@markdown - **ZIP** — среднее сжатие, удобно для Windows
#@markdown - **TAR** — быстрое сжатие, слабое

Папка_в_GDrive = os.path.join("/content/drive/MyDrive", Папка_в_GDrive)
Путь_скачивания = "/content/Work_space/DTORRENT/"
Путь_архива = "/content/Work_space/CTORRENT/"

os.makedirs(Папка_в_GDrive, exist_ok=True)
os.makedirs(Путь_скачивания, exist_ok=True)
os.makedirs(Путь_архива, exist_ok=True)

chars = re.escape(string.punctuation)

if Метод_сжатия == "ZIP":
    метод_сжатия = "zip"
    расширение = ".zip"
elif Метод_сжатия == "TAR":
    метод_сжатия = "tar"
    расширение = ".tar"
else:
    метод_сжатия = None
    расширение = ""

config = {
    'save_path': Путь_скачивания,
    'storage_mode': lt.storage_mode_t(2)
}

сессия = lt.session()
сессия.listen_on(6881, 6891)

if Метод_загрузки == "Magnet":
    if not Magnet_ссылка.strip():
        raise ValueError("Вставьте magnet-ссылку!")
    загруженный_торрент = lt.add_magnet_uri(сессия, Magnet_ссылка, config)
else:
    if torrent_params is None:
        raise ValueError("Не удалось загрузить .torrent файл")
    torrent_params.update(config)
    загруженный_торрент = сессия.add_torrent(torrent_params)

сессия.start_dht()

print("\nСобираем метаданные...")

while not загруженный_торрент.has_metadata():
    time.sleep(1)

имя = str(загруженный_торрент.status().name)

print("\nМетаданные собраны. Начинаем скачивание...\n")

последнее_обновление = 0

while not загруженный_торрент.status().is_seeding:
    статус = загруженный_торрент.status()
    прогресс = round(статус.progress * 100, 2)
    скорость_скачивания = round(статус.download_rate / 1024, 2)
    скорость_отдачи = round(статус.upload_rate / 1024, 2)

    сейчас = time.time()
    if сейчас - последнее_обновление >= 4:
        clear_output(wait=True)
        print(f"Имя: {имя}")
        print(f"Состояние: {статус.state}")
        print(f"Прогресс: {прогресс}%")
        print(f"Скорость скачивания: {скорость_скачивания} KB/s")
        print(f"Скорость отдачи: {скорость_отдачи} KB/s")
        print(f"Пиры: {статус.num_peers}")
        последнее_обновление = сейчас

    time.sleep(1)

clear_output(wait=True)

print(f"Имя: {имя}")
print("Состояние: завершено")
print("Прогресс: 100.00%")
print("Скорость скачивания: 0.00 KB/s")
print("Скорость отдачи: 0.00 KB/s")
print(f"Пиры: {загруженный_торрент.status().num_peers}")
print("✅ Скачивание завершено! Файлы в Google Drive.")

безопасное_имя = str(re.sub(r'['+chars+']', '', (имя.encode("ascii", "ignore")).decode())).replace(" ", "_")

исходная_папка = f"{Путь_скачивания}{имя}"
целевая_папка = f"{Путь_скачивания}{безопасное_имя}"
shutil.move(исходная_папка, целевая_папка)

if метод_сжатия:
    print(f"\nАрхивируем в {Метод_сжатия}...")

    имя_архива = f"{Путь_архива}{безопасное_имя}"

    shutil.make_archive(
        base_name=имя_архива,
        format=метод_сжатия,
        root_dir=целевая_папка,
        base_dir='.'
    )

    путь_к_архиву = f"{имя_архива}{расширение}"
    имя_файла_в_drive = f"{безопасное_имя}{расширение}"
else:
    print("\nБез сжатия...")
    путь_к_архиву = целевая_папка
    имя_файла_в_drive = безопасное_имя

print("\nПеренос в Google Drive...")
shutil.move(путь_к_архиву, f"{Папка_в_GDrive}/{имя_файла_в_drive}")

print("\nОчистка временных файлов...")
shutil.rmtree(целевая_папка, ignore_errors=True)

print(f"\nПуть: {Папка_в_GDrive}/{имя_файла_в_drive}")
print("\nГотово!")
print("Перед выходом: Runtime → Disconnect and delete runtime")